##### Loading the MNIST dataset

In [90]:
import torch
import torchvision
from torch import nn
from torch.nn.functional import relu, max_pool2d, log_softmax
from torchvision import datasets, transforms
from torchinfo import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

In [91]:
batch_size_train = 256
batch_size_test = 1000
lr = 1e-3
gamma = 0.7

device = torch.accelerator.current_accelerator()
device

device(type='mps')

In [92]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset1 = datasets.MNIST("data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("data", train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size_test, shuffle=True)

print(dataset1.data.shape)
print(dataset2.targets.shape)

# torch.Size([60000, 28, 28])
# torch.Size([10000])

torch.Size([60000, 28, 28])
torch.Size([10000])


In [93]:
class NeuralNetworkV1(nn.Module):
    def __init__(self):
        super(NeuralNetworkV1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, X):
        X = self.conv1(X)
        X = relu(X)
        X = self.conv2(X)
        X = relu(X)
        X = max_pool2d(X, 2)
        X = self.dropout1(X)
        X = torch.flatten(X, 1)
        X = self.fc1(X)
        X = relu(X)
        X = self.dropout2(X)
        X = self.fc2(X)
        return log_softmax(X, dim=1)


In [94]:
def train(model_train: nn.Module, loader, optimize, i, log_interval=5):
    model_train.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimize.zero_grad()
        output = model_train(data)
        loss = nn.NLLLoss()(output, target)
        loss.backward()
        optimize.step()

        if batch_idx % log_interval == 0:
            # accuracy & loss
            print(f"Accuracy: {100 * (output.argmax(dim=1) == target).float().mean().item():.2f}%")
            print(
                f"Train Epoch: {i} [{batch_idx * len(data)}/{len(loader.dataset)} ({100. * batch_idx / len(loader):.0f}%)]\tLoss: {loss.item():.6f}")

In [95]:
model = NeuralNetworkV1().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

epochs = 20
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Accuracy: 8.20%
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306466
Accuracy: 10.55%
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.301872
Accuracy: 10.55%
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.300633
Accuracy: 8.98%
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.312519
Accuracy: 7.81%
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.298841
Accuracy: 11.33%
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.287129
Accuracy: 11.33%
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.286174
Accuracy: 12.50%
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.286364
Accuracy: 12.89%
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.280546
Accuracy: 14.06%
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.292920
Accuracy: 11.33%
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.281383
Accuracy: 14.84%
Train Epoch: 1 [14080/60000 (23%)]	Loss: 2.272872
Accuracy: 12.89%
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.278741
Accuracy: 17.97%
Train Epoch: 1 [16640/60000 (28%)]	Loss: 2.273290
Accuracy: 14.45%
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.270806
Accuracy: 15.2